In [ ]:
import warnings
warnings.filterwarnings('ignore')
import glob
import os
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn import svm
import seaborn as sns; sns.set()

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Data/ETFs_main.csv')
df.head()

,Dates,CLOSE_SPY,OPEN,HIGH,LOW,VOLUME,CLOSE_GLD,CLOSE_FXY,CLOSE_T10Y2Y,CLOSE_TED,CLOSE_USO,CLOSE_UUP,CLOSE_VIX,CLOSE_VWO
0,2007-02-20,146.04,145.56,146.20,144.0,56909500.0,65.31,83.51,2.3263,0.31,48.67,25.07,10.24,40.055
1,2007-02-21,145.98,145.61,146.07,145.0,63971500.0,67.28,82.90,2.3653,0.32,49.86,25.12,10.20,39.975
2,2007-02-22,145.87,146.05,146.42,145.0,79067398.0,67.15,82.46,2.3871,0.31,50.33,25.12,10.18,40.220
3,2007-02-23,145.30,145.74,145.79,145.0,71962797.0,67.72,82.78,2.3809,0.31,50.46,25.04,10.58,40.035
4,2007-02-26,145.17,145.83,145.95,145.0,69320062.0,68.10,83.08,2.3795,0.31,50.90,25.04,11.15,39.960


In [ ]:
#기술적 지표 만들기
def moving_average(df,n):
  MA = pd.Series(df['CLOSE_SPY'].rolling(n, min_periods=n).mean(), name='MA_'+str(n))
  df = df.join(MA)
  return df

def volume_moving_average(df,n):
  VMA = pd.Series(df['VOLUME'].rolling(n, min_periods=n).mean(), name='VMA_'+str(n))
  df = df.join(VMA)
  return df

def relative_strength_index(df,n):
  i = 0
  UpI = [0]
  DoI = [0]
  while i+1 <= df.index[-1]:
    UpMove = df.loc[i+1,'HIGH'] - df.loc[i,'HIGH']
    DoMove = df.loc[i,'LOW'] - df.loc[i+1,'LOW']
    if UpMove > DoMove and UpMove > 0:
      UpD = UpMove
    else:
      UpD = 0
    UpI.append(UpD)
    if DoMove > UpMove and DoMove > 0:
      DoD = DoMove
    else:
      DoD = 0
    DoI.append(DoD)
    i = i+1
  UpI = pd.Series(UpI)
  DoI = pd.Series(DoI)
  PosDI = pd.Series(UpI.ewm(span=n,min_periods=n).mean())
  NegDI = pd.Series(DoI.ewm(span=n,min_periods=n).mean())
  RSI = pd.Series(PosDI / (PosDI + NegDI), name='RSI_'+str(n))
  df = df.join(RSI)
  return df

In [ ]:
df = moving_average(df,45)
df = volume_moving_average(df,45)
df = relative_strength_index(df,14)

In [ ]:
df = df.set_index('Dates')
df = df.dropna()
print(len(df))
df.head()

2727


,CLOSE_SPY,OPEN,HIGH,LOW,VOLUME,CLOSE_GLD,CLOSE_FXY,CLOSE_T10Y2Y,CLOSE_TED,CLOSE_USO,CLOSE_UUP,CLOSE_VIX,CLOSE_VWO,MA_45,VMA_45,RSI_14
Dates,,,,,,,,,,,,,,,,
2007-04-27,149.53,149.09,149.74,149.0,106984094.0,67.56,83.7300,2.4474,0.55,51.84,24.54,12.45,41.750,143.551556,1.106696e+08,0.670018
2007-04-30,148.29,149.64,149.74,148.0,100874203.0,67.09,83.7166,2.4361,0.57,51.24,24.49,14.22,40.935,143.601556,1.116466e+08,0.531751
2007-05-02,149.54,148.90,149.95,149.0,87129805.0,66.66,83.3800,2.4366,0.59,49.59,24.66,13.08,42.020,143.680667,1.121613e+08,0.554050
2007-05-03,150.35,149.97,150.40,149.0,87204945.0,67.49,83.1100,2.4346,0.60,49.28,24.69,13.09,42.435,143.780222,1.123421e+08,0.601028
2007-05-04,150.92,150.75,151.12,150.0,96408930.0,68.19,83.2300,2.4006,0.60,48.30,24.60,12.91,42.595,143.905111,1.128853e+08,0.665987


In [ ]:
df['target'] = df['CLOSE_SPY'].pct_change()
df['target'] = np.where(df.target > 0.0005,1,-1)
df['target'].value_counts()

 1    1375
-1    1352
Name: target, dtype: int64

In [ ]:
df['target'] = df['target'].shift(-1)
df = df.dropna()

In [ ]:
df['target'] = df.target.astype(np.int64)
y_var = df.target
x_var = df.drop(['target','OPEN','HIGH','LOW','VOLUME','CLOSE_SPY'], axis=1)
x_var.head()

,CLOSE_GLD,CLOSE_FXY,CLOSE_T10Y2Y,CLOSE_TED,CLOSE_USO,CLOSE_UUP,CLOSE_VIX,CLOSE_VWO,MA_45,VMA_45,RSI_14
Dates,,,,,,,,,,,
2007-04-27,67.56,83.7300,2.4474,0.55,51.84,24.54,12.45,41.750,143.551556,1.106696e+08,0.670018
2007-04-30,67.09,83.7166,2.4361,0.57,51.24,24.49,14.22,40.935,143.601556,1.116466e+08,0.531751
2007-05-02,66.66,83.3800,2.4366,0.59,49.59,24.66,13.08,42.020,143.680667,1.121613e+08,0.554050
2007-05-03,67.49,83.1100,2.4346,0.60,49.28,24.69,13.09,42.435,143.780222,1.123421e+08,0.601028
2007-05-04,68.19,83.2300,2.4006,0.60,48.30,24.60,12.91,42.595,143.905111,1.128853e+08,0.665987


In [ ]:
up = df[df['target']==1].target.count()
total = df.target.count()
print('up/down ratio: {0:.2f}'.format((up/total)))

up/down ratio: 0.50


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(x_var,y_var,test_size=0.3,shuffle=False,random_state=3)

train_count = y_train.count()
test_count = y_test.count()

print('train set label ratio')
print(y_train.value_counts()/train_count)
print('test set label ratio')
print(y_test.value_counts()/test_count)

train set label ratio
 1    0.509958
-1    0.490042
Name: target, dtype: float64
test set label ratio
-1    0.508557
 1    0.491443
Name: target, dtype: float64


In [ ]:
#ROC-AUC score & confusion matrix
def get_confusion_matrix(y_test,pred):
  confusion = confusion_matrix(y_test,pred)
  accuracy = accuracy_score(y_test,pred)
  precision = precision_score(y_test,pred)
  recall = recall_score(y_test,pred)
  f1 = f1_score(y_test,pred)
  roc_score = roc_auc_score(y_test,pred)
  print('confusion matrix')
  print('accuracy:{0:.4f}, precision:{1:.4f}, recall:{2:.4f}, F1:{3:.4f}, ROC-AUC:{4:.4f}'.format(accuracy,precision,recall,f1,roc_score))

In [ ]:
xgb_dis = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)
xgb_dis.fit(X_train, y_train)
xgb_pred = xgb_dis.predict(X_test)
print(xgb_dis.score(X_train,y_train))
get_confusion_matrix(y_test,xgb_pred)

0.8726415094339622
confusion matrix
accuracy:0.5012, precision:0.4920, recall:0.4602, F1:0.4756, ROC-AUC:0.5005


In [ ]:
n_estimators = range(10,200,10)

params = {
    'bootstrap': [True],
    'n_estimators':n_estimators,
    'max_depth':[4,6,8,10,12],
    'min_samples_leaf': [2, 3, 4, 5],
    'min_samples_split': [2, 4, 6, 8, 10],
    'max_features':[4]
}

In [ ]:
my_cv = TimeSeriesSplit(n_splits=5).split(X_train)

In [ ]:
clf = GridSearchCV(RandomForestClassifier(), params, cv=my_cv, n_jobs=-1)

In [ ]:
clf

GridSearchCV(cv=<generator object TimeSeriesSplit.split at 0x7f40719786d0>,
             estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [4, 6, 8, 10, 12],
                         'max_features': [4], 'min_samples_leaf': [2, 3, 4, 5],
                         'min_samples_split': [2, 4, 6, 8, 10],
                         'n_estimators': range(10, 200, 10)})

In [ ]:
clf.fit(X_train, y_train)

GridSearchCV(cv=<generator object TimeSeriesSplit.split at 0x7f40719786d0>,
             estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [4, 6, 8, 10, 12],
                         'max_features': [4], 'min_samples_leaf': [2, 3, 4, 5],
                         'min_samples_split': [2, 4, 6, 8, 10],
                         'n_estimators': range(10, 200, 10)})

In [ ]:
print('best parameter:\n',clf.best_params_)
print('best prediction:{0:.4f}'.format(clf.best_score_))

best parameter:
 {'bootstrap': True, 'max_depth': 10, 'max_features': 4, 'min_samples_leaf': 5, 'min_samples_split': 8, 'n_estimators': 10}
best prediction:0.5346


In [ ]:
pred_con = clf.predict(X_test)
accuracy_con = accuracy_score(y_test, pred_con)
print('accuracy:{0:.4f}'.format(accuracy_con))
get_confusion_matrix(y_test, pred_con)

accuracy:0.5159
confusion matrix
accuracy:0.5159, precision:0.5090, recall:0.4204, F1:0.4605, ROC-AUC:0.5143
